In [1]:
import pandas as pd 
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

data =  pd.read_csv("Data/WTS_2017.csv")
print(data.head(2))

data = data.drop(['SWIM','T1','BIKE', 'T2', 'RUN'], axis=1)

data['SWIM SEC'].replace('#VALUE!', 0, inplace=True)
data['T1 SEC'].replace('#VALUE!', 0, inplace=True)
data['BIKE SEC'].replace('#VALUE!', 0, inplace=True)
data['T2 SEC'].replace('#VALUE!', 0, inplace=True)
data['RUN SEC'].replace('#VALUE!', 0, inplace=True)
data['TOTAL TIME SEC'].replace('#VALUE!', 0, inplace=True)

data['SWIM SEC'] = data['SWIM SEC'].astype(int)
data['T1 SEC'] = data['T1 SEC'].astype(int)
data['BIKE SEC'] = data['BIKE SEC'].astype(int)
data['T2 SEC'] = data['T2 SEC'].astype(int)
data['RUN SEC'] = data['RUN SEC'].astype(int)
data['TOTAL TIME SEC'] = data['TOTAL TIME SEC'].astype(int)

Loading BokehJS ...

  ATHLETE FIRST ATHLETE LAST NATIONALITY  START NUMBER     SWIM       T1  \
0        Javier   Gomez Noya         ESP          26.0  0:18:38  0:01:02   
1        Thomas       Bishop         GBR          22.0  0:18:36  0:01:08   

      BIKE       T2      RUN POSITION TOTAL TIME    PROGRAM   LOCATION  \
0  1:00:54  0:00:36  0:31:23        1    1:52:31  Elite Men  Abu Dhabi   
1  1:00:52  0:00:36  0:31:36        2    1:52:45  Elite Men  Abu Dhabi   

  SWIM SEC T1 SEC BIKE SEC T2 SEC RUN SEC TOTAL TIME SEC  
0     1118     62     3654     36    1883           6751  
1     1116     68     3652     36    1896           6765  


In [2]:
for ind, row in data.iterrows():
    data.loc[ind, 'ATHLETE'] = data.loc[ind, 'ATHLETE FIRST'] + ' ' + data.loc[ind, 'ATHLETE LAST']
df = data[np.logical_and(data['LOCATION'] == 'Abu Dhabi', data['PROGRAM'] == 'Elite Men')]    
#print(df.keys())
locations = data['LOCATION'].unique()
#print(type(locations))
#print(locations)
programs = np.delete(data['PROGRAM'].unique(),2)
#print(programs)

# print(data['TOTAL TIME SEC'].unique())

In [3]:
#scatter plot, select location!
from ipywidgets import interact
from bokeh.io import curdoc, push_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Select, HoverTool

from numpy.random import random, normal, lognormal 
from bokeh.models.tickers import FixedTicker
from bokeh.models.formatters import TickFormatter, NumeralTickFormatter


In [6]:
df_location = data[np.logical_and(data['LOCATION'] == 'Edmonton', data['PROGRAM'] == 'Elite Men')] 
 

source = ColumnDataSource(data=dict(x= df_location['POSITION'],
                                y = df_location['TOTAL TIME SEC'],
                                name = df_location['ATHLETE'])) 

# print(source.data)
finished_len = len(df_location[df_location['TOTAL TIME SEC']!=0])
finished = df_location[df_location['TOTAL TIME SEC']!=0]

# x-ticks
x_ticks = np.array([])
result = 0

for line in range(finished_len):
    result += 1
    x_ticks = np.append(x_ticks, result)
#Hover

hover = HoverTool(tooltips=[
# hover.tooltips = OrderedDict([
    ('time', '@y'),
    ('Name', '@name'),
])


plot = figure(x_axis_label='Position', y_axis_label='Time', plot_width=900, plot_height=400)
plot.circle(x='x', y='y', source=source)
# http://bokeh.pydata.org/en/latest/docs/reference/models/formatters.html#bokeh.models.formatters.PrintfTickFormatter
plot.xaxis.ticker = FixedTicker(ticks = x_ticks)

plot.yaxis.formatter = NumeralTickFormatter(format='00:00:00')
plot.yaxis[0].ticker.desired_num_ticks = 15



def update(f = 'Edmonton', s = 'Elite Men'):
    df_location = data[np.logical_and(data['LOCATION'] == f, data['PROGRAM'] == s)] 
    source.data={'x': df_location['POSITION'], 'y': df_location['TOTAL TIME SEC']}
    finished_len = len(df_location[df_location['TOTAL TIME SEC']!=0])
    x_ticks = np.array([])
    result = 0
    for line in range(finished_len):
        result += 1
        x_ticks = np.append(x_ticks, result)
    plot.xaxis.ticker = FixedTicker(ticks = x_ticks)
    print(finished_len, x_ticks)
    push_notebook()

plot.add_tools(hover)
show(plot, notebook_handle=True)
interact(update, f=locations, s=programs )

<function __main__.update>

In [5]:
# https://www.kaggle.com/omarelgabry/a-journey-through-titanic
# https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-2-connect
# https://www.kaggle.com/anokas/outbrain-eda
# https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-sberbank